In [16]:
import os
import numpy as np
import trimesh
from scipy.spatial.transform import Rotation
base_path = 'test_1'
grasp_poses = np.load(os.path.join(base_path, 'grasp.npy'), allow_pickle=True)
point_clouds = np.load(os.path.join(base_path, 'ply.npy'), allow_pickle=True)
print(grasp_poses[0])

[[ 1.08103085e+00 -3.07364557e-02 -6.30590320e-02]
 [ 1.07983148e+00 -3.17812040e-02 -6.25277162e-02]
 [ 1.07863450e+00 -3.28224823e-02 -6.19950891e-02]
 ...
 [ 8.53616416e-01  8.95617902e-02  7.83860683e-04]
 [ 8.53008449e-01  8.86607915e-02  4.36842442e-04]
 [ 8.52400541e-01  8.77597928e-02  8.98241997e-05]]


In [12]:

class Gripper:
    gripper_depth = 59.3 / 1000
    gripper_max_width = 85.0 / 1000
    gripper_finger_width = 22.0 / 1000
    def GetGeometry(self):
        x_dim = self.gripper_max_width/2
        y_dim = self.gripper_finger_width/2
        geometry = {
            'back_plane' : np.array([
                    [-1, -1, 0],
                    [ 1, -1, 0],
                    [ 1,  1, 0],
                    [-1,  1, 0],
                ]) * [x_dim, y_dim, self.gripper_depth],
            'right_plane' : np.array([
                    [ 1, -1,  0],
                    [ 1,  1,  0],
                    [ 1,  1,  1],
                    [ 1, -1,  1],
                ]) * [x_dim, y_dim, self.gripper_depth]
        }
        geometry['left_plane'] = geometry['right_plane'] * [-1, 1, 1]
        triangles = {key:  [[0,1,2],[2,3,0], [1, 0, 2], [3, 2, 0]] for key in geometry}
        return geometry, triangles
gripper = Gripper()
gripper_geometry, triangles = gripper.GetGeometry()
gripper_meshes = { key : trimesh.Trimesh(gripper_geometry[key], triangles[key]) for key in gripper_geometry}

In [13]:
def PoseToMatrix(pose):
    m = np.zeros((4,4))
    m[0:3, 0:3] = Rotation.from_rotvec(pose[3:]).as_matrix()
    m[0:3, 3] = pose[:3]
    return m

In [14]:
pc = trimesh.PointCloud(point_clouds[0], [255, 0, 0, 0], metadata={'point_size':10000})
actual_pose = grasp_poses[0]
tf = PoseToMatrix(actual_pose)
sc = trimesh.Scene(pc)
for key in gripper_meshes:
    sc.add_geometry(gripper_meshes[key], transform=tf)

ValueError: could not broadcast input array from shape (14983,3,3) into shape (3,3)